In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/project1/

/content/drive/My Drive/project1


In [ ]:
import cv2,os
data_path='/content/drive/MyDrive/project1/datasets/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'Non Glaucoma': 0, 'Glaucoma': 1}
['Non Glaucoma', 'Glaucoma']
[0, 1]


In [ ]:
img_size=128
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [ ]:
new_target.shape

(705, 2)

In [ ]:
data.shape

(705, 128, 128, 3)

In [ ]:
data.shape[1:]

(128, 128, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 200)     5600      
                                                                 
 activation (Activation)     (None, 126, 126, 200)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 200)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 100)       180100    
                                                                 
 activation_1 (Activation)   (None, 61, 61, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 100)      0         
 2D)                                                    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score , precision_score , confusion_matrix, mean_squared_error
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
train_data.shape

(634, 128, 128, 3)

In [ ]:
train_target.shape

(634, 2)

In [ ]:
history=model.fit(train_data,train_target,epochs=5,validation_split=0.2)

Epoch 1/5
16/16 [==============================] - 14s 252ms/step - loss: 0.8822 - accuracy: 0.6548 - val_loss: 0.5747 - val_accuracy: 0.6929
Epoch 2/5
16/16 [==============================] - 3s 172ms/step - loss: 0.4131 - accuracy: 0.8067 - val_loss: 0.4528 - val_accuracy: 0.7717
Epoch 3/5
16/16 [==============================] - 3s 171ms/step - loss: 0.3169 - accuracy: 0.8797 - val_loss: 0.3355 - val_accuracy: 0.8819
Epoch 4/5
16/16 [==============================] - 3s 169ms/step - loss: 0.2547 - accuracy: 0.8876 - val_loss: 0.2865 - val_accuracy: 0.8898
Epoch 5/5
16/16 [==============================] - 3s 175ms/step - loss: 0.2671 - accuracy: 0.9132 - val_loss: 0.3162 - val_accuracy: 0.8898


In [ ]:
model.save('model.h5')

In [ ]:
# Convert the model.
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpb1ad3yay/assets


In [ ]:
y_pred=model.predict(test_data) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_target, axis=1)
print('Test confusion matrix of cnn = ',confusion_matrix(y_test, y_pred))
print('Test recall of cnn = ',recall_score(y_test, y_pred, average=None))
print('Test precision of cnn = ',precision_score(y_test, y_pred, average=None))
print('Test rms of cnn = ',mean_squared_error(y_test, y_pred, squared=False))


Test confusion matrix of cnn =  [[28  3]
 [ 7 33]]
Test recall of cnn =  [0.90322581 0.825     ]
Test precision of cnn =  [0.8        0.91666667]
Test rms of cnn =  0.37529331252040077


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# plot the training loss and accuracy
N = 5
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy FOR CNN")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")

# ***DenseNet121***

```
# This is formatted as code
```



In [ ]:
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPool2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.densenet import DenseNet121

In [ ]:
tr = DenseNet121(weights='imagenet', include_top=False, input_shape=(100,100,3))

In [ ]:
model = Sequential()
model.add(tr)
model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,train_target,epochs=5,validation_split=0.2)

In [ ]:
model.save('DenseNet121_model.h5')

In [ ]:
y_pred=model.predict(test_data) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_target, axis=1)
print('Test confusion matrix of DenseNet121 = ',confusion_matrix(y_test, y_pred))
print('Test recall of DenseNet121 = ',recall_score(y_test, y_pred, average=None))
print('Test precision of DenseNet121 = ',precision_score(y_test, y_pred, average=None))
print('Test rms of DenseNet121 = ',mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# plot the training loss and accuracy
N = 5
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy FOR DenseNet121")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("DenseNet121_Model")

**vgg16** 

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))

In [ ]:
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,train_target,epochs=5,validation_split=0.2)

In [ ]:
model.save('vgg16_model.h5')

In [ ]:
y_pred=model.predict(test_data) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_target, axis=1)
print('Test confusion matrix of vgg16 = ',confusion_matrix(y_test, y_pred))
print('Test recall of vgg16= ',recall_score(y_test, y_pred, average=None))
print('Test precision of vgg16 = ',precision_score(y_test, y_pred, average=None))
print('Test rms of vgg16 = ',mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# plot the training loss and accuracy
N = 5
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy FOR vgg16")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("vgg16_Model")

# ***resnet50***

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
ResNet = ResNet50(weights='imagenet', include_top=False, input_shape=(100,100,3))

In [ ]:
model = Sequential()
model.add(ResNet)
model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,train_target,epochs=5,validation_split=0.2)

In [ ]:
model.save('ResNet_model.h5')

In [ ]:
y_pred=model.predict(test_data) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_target, axis=1)
print('Test confusion matrix of ResNet50 = ',confusion_matrix(y_test, y_pred))
print('Test recall of ResNet50= ',recall_score(y_test, y_pred, average=None))
print('Test precision of ResNet50 = ',precision_score(y_test, y_pred, average=None))
print('Test rms of ResNet50 = ',mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# plot the training loss and accuracy
N = 5
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy FOR ResNet")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("ResNet_Model")